In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "G:\\Code\\Node\\TemporalPoliMap\\secrets\\bigquery_secrets_file.json"
from google.cloud import bigquery
import csv
import uuid
import pandas as pd

In [2]:
client = bigquery.Client()

In [3]:
def getData(cty_code):
    query="""
    SELECT Source, Target, Year, QuadClass, Count(EventCode) as ECount, SUM(NumSources) as NSources, MAX(NumSources) as MaxSources, AVG(NumSources) as AvgSources, STDDEV(NumSources) as StdDevSources
    FROM
    (
    SELECT  Actor1CountryCode as Source, Actor2CountryCode as Target, Year, QuadClass,EventCode,NumSources
    FROM [gdelt-bq:full.events]
    WHERE (Actor1CountryCode = '{cty_code}') AND (Actor1CountryCode != Actor2CountryCode)
    AND (Actor1Type1Code = "GOV" OR Actor1Type2Code = "GOV" OR Actor1Type3Code = "GOV")
    ),
    (
    SELECT  Actor2CountryCode as Source, Actor1CountryCode as Target, Year, QuadClass,EventCode,NumSources
    FROM [gdelt-bq:full.events]
    WHERE (Actor2CountryCode = '{cty_code}') AND (Actor2CountryCode != Actor1CountryCode)
    AND (Actor2Type1Code = "GOV" OR Actor2Type2Code = "GOV" OR Actor2Type3Code = "GOV")
    )
    GROUP BY Source, Target, Year, QuadClass ORDER BY MaxSources DESC;
    """.format(cty_code=cty_code)
    
    print("Downloading data for: ", cty_code)
    query_job = client.run_async_query(str(uuid.uuid4()), query)
    query_job.begin()
    query_job.result()  # Wait for job to complete.
    # [END run_query]

    # [START print_results]
    destination_table = query_job.destination
    destination_table.reload()

    results = [row for row in destination_table.fetch_data()]
    columns = [schemaField.name for schemaField in destination_table.schema]
    df = pd.DataFrame(results, columns=columns)
    outputfile = "../data/{}.csv".format(cty_code)
    df.to_csv(outputfile, index=False)
    print("Saved {} (rows, columns) to {}".format(df.shape, outputfile))

In [4]:
codes = ["USA","CHN","IND","AFG","BRA","FRA","GBR","IRN","IRQ","PAK","ISR","RUS","ZAF","AUS", "GRC"]
print("Downloading data for following countries: ", codes)
for code in codes:
    getData(code)

Saved (19284, 9) (rows, columns) to ../data/USA.csv
Saved (10386, 9) (rows, columns) to ../data/CHN.csv
Saved (4645, 9) (rows, columns) to ../data/IND.csv
Saved (4326, 9) (rows, columns) to ../data/AFG.csv
Saved (3823, 9) (rows, columns) to ../data/BRA.csv
Saved (10889, 9) (rows, columns) to ../data/FRA.csv
Saved (12345, 9) (rows, columns) to ../data/GBR.csv
Saved (7603, 9) (rows, columns) to ../data/IRN.csv
Saved (5941, 9) (rows, columns) to ../data/IRQ.csv
Saved (5612, 9) (rows, columns) to ../data/PAK.csv
Saved (7826, 9) (rows, columns) to ../data/ISR.csv
Saved (10928, 9) (rows, columns) to ../data/RUS.csv
Saved (5571, 9) (rows, columns) to ../data/ZAF.csv
Saved (6475, 9) (rows, columns) to ../data/AUS.csv
Saved (3922, 9) (rows, columns) to ../data/GRC.csv
